In [ ]:
import csv
import datetime as dt
import json
import os
import statistics
import time
import numpy as np
import pandas as pd
import requests

In [ ]:
pd.options.display.max_colwidth = 9999

In [ ]:
pd.set_option("max_columns",100)

In [ ]:
def get_request(url, parameters = None):
  try:
    response = requests.get(url = url,params = parameters)
  except SSLError as s:
    print('SSL Error:', s)

    for i in range(5, 0, -1):
      print('\rWaiting... ({})'.format(i), end='')
      time.sleep(1)
    print('\rRetrying.' + ' '*10)
    return get_request(url, parameters)

  if response:
    return response.json()
  else:
    print('No response, waiting 10 seconds...')
    time.sleep(10)
    print('Retrying.')
    return get_request(url, parameters)

In [ ]:
url  = "https://steamspy.com/api.php"
i = 0
stall = pd.DataFrame()
while True:
  parameters = {"request": "all","page":i}
  json_data = get_request(url, parameters=parameters)
  steamspy_all = pd.DataFrame.from_dict(json_data, orient= "index")
  app_list = steamspy_all#[['appid', 'name']]
  stall = pd.concat([stall,app_list])
  stall = stall.reset_index(drop=True)
  if app_list.shape[0]<1000:
    break
  i+=1

In [ ]:
stall.shape

(55387, 17)

In [ ]:
stall.head()

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,ccu
0,570,Dota 2,Valve,Valve,,1474975,299641,0,"100,000,000 .. 200,000,000",35374,1463,914,730,0,0,0,581058
1,730,Counter-Strike: Global Offensive,"Valve, Hidden Path Entertainment",Valve,,5750512,765101,0,"50,000,000 .. 100,000,000",30034,812,7043,278,0,0,0,746122
2,578080,PUBG: BATTLEGROUNDS,"KRAFTON, Inc.","KRAFTON, Inc.",,1152775,895096,0,"50,000,000 .. 100,000,000",23199,886,7355,230,0,0,0,313078
3,1063730,New World,Amazon Games,Amazon Games,,154765,73837,0,"50,000,000 .. 100,000,000",7495,1172,3058,997,3999,3999,0,18186
4,440,Team Fortress 2,Valve,Valve,,822203,56623,0,"50,000,000 .. 100,000,000",9205,1422,428,508,0,0,0,118412


In [ ]:
stall = stall.sort_values('ccu',ascending=False).reset_index(drop=True)

In [ ]:
stall

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,ccu
0,730,Counter-Strike: Global Offensive,"Valve, Hidden Path Entertainment",Valve,,5750512,765101,0,"50,000,000 .. 100,000,000",30034,812,7043,278,0,0,0,746122
1,570,Dota 2,Valve,Valve,,1474975,299641,0,"100,000,000 .. 200,000,000",35374,1463,914,730,0,0,0,581058
2,578080,PUBG: BATTLEGROUNDS,"KRAFTON, Inc.","KRAFTON, Inc.",,1152775,895096,0,"50,000,000 .. 100,000,000",23199,886,7355,230,0,0,0,313078
3,1172470,Apex Legends,Respawn Entertainment,Electronic Arts,,413737,66109,0,"20,000,000 .. 50,000,000",5525,742,636,407,0,0,0,243474
4,1599340,Lost Ark,Smilegate RPG,Amazon Games,,123773,43908,0,"20,000,000 .. 50,000,000",2957,872,718,166,0,0,0,203513
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55382,1697780,Life Can Be Amazing,Double Peach Productions,Little Black Book Entertainment,,3,5,0,"0 .. 20,000",0,0,0,0,799,799,0,0
55383,739850,Goblins Keep Coming - Tower Defense,Los Exaltados,Los Exaltados,,25,7,0,"0 .. 20,000",0,0,0,0,199,199,0,0
55384,1742750,Delta Zero,Tyler M Richardson,Tyler M Richardson,,3,0,0,"0 .. 20,000",0,0,0,0,999,999,0,0
55385,977640,Happy toys,bdme,bdme,,98,50,0,"0 .. 20,000",4,0,4,0,79,199,60,0


In [ ]:
stall = stall.drop(columns=['score_rank','owners','average_forever','average_2weeks','median_forever','median_2weeks','initialprice','discount'])

In [ ]:
stall

,appid,name,developer,publisher,positive,negative,userscore,price,ccu
0,730,Counter-Strike: Global Offensive,"Valve, Hidden Path Entertainment",Valve,5750512,765101,0,0,746122
1,570,Dota 2,Valve,Valve,1474975,299641,0,0,581058
2,578080,PUBG: BATTLEGROUNDS,"KRAFTON, Inc.","KRAFTON, Inc.",1152775,895096,0,0,313078
3,1172470,Apex Legends,Respawn Entertainment,Electronic Arts,413737,66109,0,0,243474
4,1599340,Lost Ark,Smilegate RPG,Amazon Games,123773,43908,0,0,203513
...,...,...,...,...,...,...,...,...,...
55382,1697780,Life Can Be Amazing,Double Peach Productions,Little Black Book Entertainment,3,5,0,799,0
55383,739850,Goblins Keep Coming - Tower Defense,Los Exaltados,Los Exaltados,25,7,0,199,0
55384,1742750,Delta Zero,Tyler M Richardson,Tyler M Richardson,3,0,0,999,0
55385,977640,Happy toys,bdme,bdme,98,50,0,79,0


In [ ]:
path = '/content/drive/MyDrive/game_recomender/steamspy_data.csv'
with open(path, 'w', encoding = 'utf-8-sig') as f:
  stall.to_csv(f)

In [ ]:
stall.shape

(55387, 8)

In [ ]:
url = "http://store.steampowered.com/api/appdetails/"
parameters = {"appids": 730}

resp = get_request(url,parameters)
k = resp['730']

In [ ]:
k

{'data': {'about_the_game': "Counter-Strike: Global Offensive (CS: GO) expands upon the team-based action gameplay that it pioneered when it was launched 19 years ago.<br />\r\n<br />\r\nCS: GO features new maps, characters, weapons, and game modes, and delivers updated versions of the classic CS content (de_dust2, etc.).<br />\r\n<br />\r\n&quot;Counter-Strike took the gaming industry by surprise when the unlikely MOD became the most played online PC action game in the world almost immediately after its release in August 1999,&quot; said Doug Lombardi at Valve. &quot;For the past 12 years, it has continued to be one of the most-played games in the world, headline competitive gaming tournaments and selling over 25 million units worldwide across the franchise. CS: GO promises to expand on CS' award-winning gameplay and deliver it to gamers on the PC as well as the next gen consoles and the Mac.&quot;",
  'achievements': {'highlighted': [{'name': 'Someone Set Up Us The Bomb',
     'path'

In [ ]:
lis = ['steam_appid','categories','genres','header_image','metacritic','short_description']
dic = {}
for i in lis:
  dic[i] = k['data'][i]
dic

{'categories': [{'description': 'Multi-player', 'id': 1},
  {'description': 'Steam Achievements', 'id': 22},
  {'description': 'Full controller support', 'id': 28},
  {'description': 'Steam Trading Cards', 'id': 29},
  {'description': 'Steam Workshop', 'id': 30},
  {'description': 'In-App Purchases', 'id': 35},
  {'description': 'Valve Anti-Cheat enabled', 'id': 8},
  {'description': 'Stats', 'id': 15},
  {'description': 'Remote Play on Phone', 'id': 41},
  {'description': 'Remote Play on Tablet', 'id': 42},
  {'description': 'Remote Play on TV', 'id': 43}],
 'genres': [{'description': 'Action', 'id': '1'},
  {'description': 'Free to Play', 'id': '37'}],
 'header_image': 'https://cdn.akamai.steamstatic.com/steam/apps/730/header.jpg?t=1641233427',
 'metacritic': {'score': 83,
  'url': 'https://www.metacritic.com/game/pc/counter-strike-global-offensive?ftag=MCD-06-10aaa1f'},
 'short_description': 'Counter-Strike: Global Offensive (CS: GO) expands upon the team-based action gameplay that 

In [ ]:
dic['metacritic']

{'score': 83,
 'url': 'https://www.metacritic.com/game/pc/counter-strike-global-offensive?ftag=MCD-06-10aaa1f'}

In [ ]:
dic['metacritic'] = dic['metacritic']
dic['genres'] = dic['genres'][0]['description']
l = []
for i in dic['categories']:
  l.append(i['description'])
dic['categories'] = l
dic

{'categories': ['Multi-player',
  'Steam Achievements',
  'Full controller support',
  'Steam Trading Cards',
  'Steam Workshop',
  'In-App Purchases',
  'Valve Anti-Cheat enabled',
  'Stats',
  'Remote Play on Phone',
  'Remote Play on Tablet',
  'Remote Play on TV'],
 'genres': 'Action',
 'header_image': 'https://cdn.akamai.steamstatic.com/steam/apps/730/header.jpg?t=1641233427',
 'metacritic': {'score': 83,
  'url': 'https://www.metacritic.com/game/pc/counter-strike-global-offensive?ftag=MCD-06-10aaa1f'},
 'short_description': 'Counter-Strike: Global Offensive (CS: GO) expands upon the team-based action gameplay that it pioneered when it was launched 19 years ago. CS: GO features new maps, characters, weapons, and game modes, and delivers updated versions of the classic CS content (de_dust2, etc.).',
 'steam_appid': 730}

In [ ]:
v = pd.DataFrame.from_dict(dic,orient='index')
v = v.transpose()


In [ ]:
def steama(appid):
  url = "http://store.steampowered.com/api/appdetails/"
  parameters = {"appids": appid}

  resp = get_request(url,parameters)
  k = resp[str(appid)]
  lis = ['steam_appid','categories','genres','header_image','short_description']
  dic = {}
  #print(appid)
  #print(appid)
  #print(k['metacritic'])
  for i in lis:
    try:
      dic[i] = k['data'][i]
    except:
      dic[i] = np.nan
  try:
    h = k['data']['metacritic']['score']
  except:
    h = np.nan
  dic['meta'] = h
  try:
    dic['genres'] = dic['genres'][0]['description']
  except:
    print(dic['genres'])
    pass
  l = []
  try:
    for i in dic['categories']:
      l.append(i['description'])
    dic['categories'] = l
  except:
    pass
  v = pd.DataFrame.from_dict(dic,orient='index')
  v = v.transpose()
  return v

In [ ]:
stall = pd.read_csv('/content/drive/MyDrive/game_recomender/steamspy_data.csv')

In [ ]:
stall

,Unnamed: 0,appid,name,developer,publisher,positive,negative,price,ccu
0,0,730,Counter-Strike: Global Offensive,"Valve, Hidden Path Entertainment",Valve,5750512,765101,0.0,746122
1,1,570,Dota 2,Valve,Valve,1474975,299641,0.0,581058
2,2,578080,PUBG: BATTLEGROUNDS,"KRAFTON, Inc.","KRAFTON, Inc.",1152775,895096,0.0,313078
3,3,1172470,Apex Legends,Respawn Entertainment,Electronic Arts,413737,66109,0.0,243474
4,4,1599340,Lost Ark,Smilegate RPG,Amazon Games,123773,43908,0.0,203513
...,...,...,...,...,...,...,...,...,...
55382,55382,1697780,Life Can Be Amazing,Double Peach Productions,Little Black Book Entertainment,3,5,799.0,0
55383,55383,739850,Goblins Keep Coming - Tower Defense,Los Exaltados,Los Exaltados,25,7,199.0,0
55384,55384,1742750,Delta Zero,Tyler M Richardson,Tyler M Richardson,3,0,999.0,0
55385,55385,977640,Happy toys,bdme,bdme,98,50,79.0,0


In [ ]:
steamapi = pd.DataFrame()
for appid in stall['appid'][0:10]:
  t = steama(appid)
  steamapi = pd.concat([steamapi,t])
steamapi

nan


,steam_appid,categories,genres,header_image,short_description,meta
0,730,"[Multi-player, Steam Achievements, Full controller support, Steam Trading Cards, Steam Workshop, In-App Purchases, Valve Anti-Cheat enabled, Stats, Remote Play on Phone, Remote Play on Tablet, Remote Play on TV]",Action,https://cdn.akamai.steamstatic.com/steam/apps/730/header.jpg?t=1641233427,"Counter-Strike: Global Offensive (CS: GO) expands upon the team-based action gameplay that it pioneered when it was launched 19 years ago. CS: GO features new maps, characters, weapons, and game modes, and delivers updated versions of the classic CS content (de_dust2, etc.).",83
0,570,"[Multi-player, Co-op, Steam Trading Cards, Steam Workshop, SteamVR Collectibles, In-App Purchases, Valve Anti-Cheat enabled]",Action,https://cdn.akamai.steamstatic.com/steam/apps/570/header.jpg?t=1650611880,"Every day, millions of players worldwide enter battle as one of over a hundred Dota heroes. And no matter if it's their 10th hour of play or 1,000th, there's always something new to discover. With regular updates that ensure a constant evolution of gameplay, features, and heroes, Dota 2 has taken on a life of its own.",90
0,578080,"[Multi-player, PvP, Online PvP, Stats, Remote Play on Phone, Remote Play on Tablet]",Action,https://cdn.akamai.steamstatic.com/steam/apps/578080/header.jpg?t=1657765663,"Play PUBG: BATTLEGROUNDS for free. Land on strategic locations, loot weapons and supplies, and survive to become the last team standing across various, diverse Battlegrounds. Squad up and join the Battlegrounds for the original Battle Royale experience that only PUBG: BATTLEGROUNDS can offer.",NaN
0,1172470,"[Multi-player, PvP, Online PvP, Co-op, Online Co-op, Steam Achievements, Full controller support, Steam Trading Cards, In-App Purchases]",Action,https://cdn.akamai.steamstatic.com/steam/apps/1172470/header.jpg?t=1657217209,"Apex Legends is the award-winning, free-to-play Hero Shooter from Respawn Entertainment. Master an ever-growing roster of legendary characters with powerful abilities, and experience strategic squad play and innovative gameplay in the next evolution of Hero Shooter and Battle Royale.",88
0,NaN,NaN,NaN,NaN,NaN,NaN
0,1446780,"[Single-player, Multi-player, Co-op, Online Co-op, Steam Achievements, Full controller support, Steam Trading Cards, Steam Cloud]",Action,https://cdn.akamai.steamstatic.com/steam/apps/1446780/header.jpg?t=1656665891,"Rise to the challenge and join the hunt! In Monster Hunter Rise, the latest installment in the award-winning and top-selling Monster Hunter series, you’ll become a hunter, explore brand new maps and use a variety of weapons to take down fearsome monsters as part of an all-new storyline.",87
0,271590,"[Single-player, Multi-player, PvP, Online PvP, Co-op, Online Co-op, Steam Achievements, Full controller support, Remote Play on Phone, Remote Play on Tablet, Remote Play on TV]",Action,https://cdn.akamai.steamstatic.com/steam/apps/271590/header.jpg?t=1618856444,"Grand Theft Auto V for PC offers players the option to explore the award-winning world of Los Santos and Blaine County in resolutions of up to 4k and beyond, as well as the chance to experience the game running at 60 frames per second.",96
0,440,"[Multi-player, Cross-Platform Multiplayer, Steam Achievements, Steam Trading Cards, Captions available, Steam Workshop, In-App Purchases, Partial Controller Support, Valve Anti-Cheat enabled, Stats, Includes level editor, Commentary available, Remote Play on Phone, Remote Play on Tablet]",Action,https://cdn.akamai.steamstatic.com/steam/apps/440/header.jpg?t=1592263852,"Nine distinct classes provide a broad range of tactical abilities and personalities. Constantly updated with new game modes, maps, equipment and, most importantly, hats!",92
0,346110,"[Single-player, Multi-player, MMO, Co-op, Steam Achievements, Full controller support, Steam Workshop, Steam Cloud, Valve Anti-Cheat enabled, Includes level editor, Remote Play on Tablet]

In [ ]:
for i in range(10):
  initial = i*300
  final = (i+1)*300
  steamapi = pd.DataFrame()
  for j in range(initial,final):
    appid = stall['appid'][j]
    t = steama(appid)
    steamapi = pd.concat([steamapi,t])
    time.sleep(1)
  print(initial)
  print(final)
  path = '/content/drive/MyDrive/game_recomender/steamapi_'+ str(initial) + '_' + str(final)+'.csv'
  with open(path, 'w', encoding = 'utf-8-sig') as f:
    steamapi.to_csv(f)


nan
0
300
No response, waiting 10 seconds...
Retrying.
No response, waiting 10 seconds...
Retrying.
No response, waiting 10 seconds...
Retrying.
nan
nan
No response, waiting 10 seconds...
Retrying.
No response, waiting 10 seconds...
Retrying.
No response, waiting 10 seconds...
Retrying.
nan
nan
300
600
No response, waiting 10 seconds...
Retrying.
No response, waiting 10 seconds...
Retrying.
No response, waiting 10 seconds...
Retrying.
No response, waiting 10 seconds...
Retrying.
nan
600
900
No response, waiting 10 seconds...
Retrying.
No response, waiting 10 seconds...
Retrying.
No response, waiting 10 seconds...
Retrying.
No response, waiting 10 seconds...
Retrying.
nan
No response, waiting 10 seconds...
Retrying.
No response, waiting 10 seconds...
Retrying.
No response, waiting 10 seconds...
Retrying.
900
1200
No response, waiting 10 seconds...
Retrying.
No response, waiting 10 seconds...
Retrying.
No response, waiting 10 seconds...
Retrying.
No response, waiting 10 seconds...
Retryi

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/game_recomender/data3.csv')

In [ ]:
df.head()

,Unnamed: 0,Unnamed: 0.1,steam_appid,categories,genres,header_image,name,developer,publisher,price,ccu,score
0,0,0,1569040,"['Single-player', 'Multi-player', 'Steam Achievements', 'Steam Cloud']",Simulation,https://cdn.akamai.steamstatic.com/steam/apps/1569040/header.jpg?t=1649351903,Football Manager 2022,Sports Interactive,SEGA,5499.0,71267,92.0
1,1,1,493520,"['Multi-player', 'Co-op', 'Online Co-op', 'Steam Trading Cards', 'Partial Controller Support']",Action,https://cdn.akamai.steamstatic.com/steam/apps/493520/header.jpg?t=1655707833,GTFO,10 Chambers,10 Chambers,2999.0,16161,78.0
2,2,2,1824220,"['Multi-player', 'PvP', 'Online PvP', 'Cross-Platform Multiplayer', 'Steam Achievements', 'Steam Trading Cards', 'In-App Purchases', 'Partial Controller Support']",Action,https://cdn.akamai.steamstatic.com/steam/apps/1824220/header.jpg?t=1655477182,Chivalry 2,Torn Banner Studios,Tripwire Presents,2399.0,15457,82.0
3,3,3,1621690,"['Single-player', 'Multi-player', 'Co-op', 'Online Co-op', 'Full controller support', 'Steam Cloud']",Adventure,https://cdn.akamai.steamstatic.com/steam/apps/1621690/header_alt_assets_4.jpg?t=1655385001,Core Keeper,Pugstorm,Fireshine Games,1169.0,10349,90.0
4,4,4,1462040,"['Single-player', 'Steam Achievements', 'Full controller support', 'Steam Trading Cards', 'Steam Cloud']",Action,https://cdn.akamai.steamstatic.com/steam/apps/1462040/header.jpg?t=1655449232,FINAL FANTASY VII REMAKE INTERGRADE,Square Enix,Square Enix,4969.0,10326,90.0
